In [ ]:
#! pip install transformers datasets evaluate

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device.type == "cuda":
    print("GPU: ", torch.cuda.get_device_name(0))
else:
    print("CPU")

In [ ]:
import pandas as pd
from datasets import load_dataset

In [ ]:
dataset_train = load_dataset("csv", data_files="amazon Food Reviews 100k Dataset.csv", 
                       split="train[:1%]")

dataset_eval = load_dataset("csv", data_files="amazon Food Reviews 100k Dataset.csv", 
                       split="train[1%:11%]")

In [ ]:
dataset_train = dataset_train.rename_column("Review", "text")\
    .rename_column("Rating", "label")\
    .remove_columns(['Id'])

dataset_eval = dataset_eval.rename_column("Review", "text")\
    .rename_column("Rating", "label")\
    .remove_columns(['Id'])

## Tokenizer

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
# apply tokenizer
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=50)


tokenized_dataset_train = dataset_train.map(tokenize_function, batched=True)
tokenized_dataset_eval = dataset_eval.map(tokenize_function, batched=True)

In [ ]:
# https://download.pytorch.org/whl/cu116
# !pip3 install torch torchvision torchaudio --extra-index-url 

## Process labels

In [ ]:
def minus_1(example):
    example["label"] = example["label"] - 1
    return example

In [ ]:
tokenized_dataset_train = tokenized_dataset_train.map(minus_1)
tokenized_dataset_eval = tokenized_dataset_eval.map(minus_1)

In [ ]:
len(tokenized_dataset_train.to_pandas().loc[0,'input_ids'])
# how to reduce padding ?

## Load the model

Start by loading your model and specify the number of expected labels.

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

In [ ]:
#model = model.to('cuda')

[Training hyperparameters](https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/trainer#transformers.TrainingArguments) - Specify where to save the checkpoints from your training.
specify the evaluation_strategy parameter in your training arguments to report the evaluation metric at the end of each epoch:

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer", 
                                  optim="adamw_torch", 
                                  save_strategy='epoch',
                                  evaluation_strategy="epoch"
                                 )

## Metric

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

## Trainer

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_eval,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

## Evaluation

using whole eval dataset

In [ ]:
trainer.evaluate()

Using validation dataset

In [ ]:
dataset_validation = load_dataset("csv", data_files="amazon Food Reviews 100k Dataset.csv", 
                       split="train[30%:31%]")

In [ ]:
dataset_validation = dataset_validation.rename_column("Review", "text")\
    .rename_column("Rating", "label")\
    .remove_columns(['Id'])

In [ ]:
tokenized_dataset_validation = dataset_validation.map(tokenize_function, batched=True)

In [ ]:
tokenized_dataset_validation = tokenized_dataset_validation.map(minus_1)

In [ ]:
predictions = trainer.predict(tokenized_dataset_validation)

In [ ]:
predictions.metrics

## Prediction

In [ ]:
dataset_new = load_dataset("csv", data_files="amazon Food Reviews 100k Dataset.csv", 
                       split="train[3%:4%]")

In [ ]:
dataset_new = dataset_new.rename_column("Review", "text").remove_columns(['Id', 'Rating'])

In [ ]:
tokenized_dataset_new = dataset_new.map(tokenize_function, batched=True)

In [ ]:
logits_new = trainer.predict(tokenized_dataset_new)

In [ ]:
predictions = np.argmax(logits_new.predictions, axis=-1)

In [ ]:
predictions[:4]